### GPS (Gas Prediction System) by GPT (Gas Protection System)

Esse é um arquivo python notebook (.ipynb) para uma compreensão de dados para a parceria com a Compass. São utilizadas as bibliotecas pandas, numpy, matplotlib e seaborn. É feito o carregamento e concatenação simultânea de cada tabela (.csv) contendo de dados dos meses. É utilizada uma função para descrever a estatística descritiva de diferentes colunas da tabela. Em seguida foram criados gráficos para diferentes análises da tabela.

In [2]:
# Carrega biblioteca pandas
import pandas as pnd
# Carrega biblioteca numpy
import numpy as np
# Carrega biblioteca matplotlib
import matplotlib.pyplot as plt
# Carrega biblioteca seaborn
import seaborn as sb

##### Carregamento e Concatenação de Todas Tabelas

In [ ]:
# Concatena diferentes tabelas com cada mês (dados)
# Em seguida salva todos os dados em um único DataFrame chamado df
listas = ['month_2.csv', 'month_3.csv', 'month_4.csv', 'month_5.csv', 'month_6.csv']
df = []
for arquivo in listas:
    df += [pnd.read_csv(arquivo)]

# Concatena todos os dataframes em um único dataframe chamado df
df = pnd.concat(df)

# Chama o dataframe contido na variável chamada df
df

##### Carrega Dados Cadastrais

In [ ]:
# Carrega e salva dadosCadastrais como dadosCadastrais
dadosCadastrais = pnd.read_csv('informacao_cadastral.csv')

# Chama o dataframe contido na variável dadoCadastrais
dadosCadastrais

##### Normalização Coluna 'cidade'

In [ ]:
# Carrega diferentes possibilidades para a coluna 'cidade'
# Antes da normalização
dadosCadastrais['cidade'].unique()

In [ ]:
# Substituindo "GRAVATAI" por "GRAVATAÍ" na coluna "cidade"
dadosCadastrais['cidade'] = dadosCadastrais['cidade'].replace('GRAVATAI', 'GRAVATAÍ')

# Substituindo "SAO LEOPOLDO" por "SÃO LEOPOLDO" na coluna "cidade"
dadosCadastrais['cidade'] = dadosCadastrais['cidade'].replace('SAO LEOPOLDO', 'SÃO LEOPOLDO')

# Verificando entradas na coluna "cidade"
dadosCadastrais['cidade'].unique()

#### Estatística Descritiva

Nessa sessão é criada uma função que estende a função describe da biblioteca 'pandas'. Adiciona variância, média e moda.

##### Cria Função de Estatística Descritiva Completa

In [7]:
# Cria uma função extendida de descrever (estatísticas descritivas)
def describeExtended(data):
    description = data.describe()

    # Adiciona a mediana, variância e moda para a função describe	
    description.loc['var'] = data.var()
    description.loc['median'] = data.median()
    description.loc['mode'] = data.mode().iloc[0]
    return description

##### Utilização da Função Describe Nova Para 'pulseCount', 'meterIndex' e 'initialIndex'

In [ ]:
# Estatística descritiva pulseCount
print(describeExtended(df.pulseCount))

In [ ]:
# Estatística descritiva meterIndex
print(describeExtended(df.meterIndex))

In [ ]:
# Estatística descritiva initialIndex
print(describeExtended(df.initialIndex))

#### Pré-Processamento de Dados

##### Merge de Dataframes

Nessa seção são feitas algumas alterações básicas e geradas uma nova dataframe a partir do que foi filtrado, organizado (ordenar por data), e calculadas as diferenças entre valores.

In [ ]:
# Filtra os usuários que estão consumindo gás (operacionais) e por seu código de cliente
usuariosUnicos = dadosCadastrais[dadosCadastrais.situacao == 'CONSUMINDO GÁS']['clientCode'].unique() 
# Organiza os dados dos usuários filtrados pela data
mesFiltrado = df[df['clientCode'].isin(usuariosUnicos)].sort_values(by='datetime') 
# Filtra meterSN diferente de '>N<A'
mesFiltrado = mesFiltrado[mesFiltrado['meterSN'] != '>N<A']
# Cria uma nova variável mesFiltrado, agrupa por meterSN e clientCode e seleciona a primeira linha
resultado = mesFiltrado.groupby(['meterSN', 'clientCode']).first() 
# Seleciona a coluna pulseCount
resultado = resultado[['pulseCount']] 
# Cria um novo dataframe com a coluna pulseCountInicial
final = pnd.DataFrame({'pulseCountInicial': resultado.pulseCount}) 
# Junta os dataframes
merged_df = pnd.merge(df, final, on=['meterSN', 'clientCode'], how='left') 
# Calcula a diferença entre pulseCount e pulseCountInicial
merged_df['pulseCount'] = merged_df['pulseCount'] - merged_df['pulseCountInicial'] 
#Mostra o dataframe 'merged_df'
merged_df

#### Gráficos

Criação de diversos gráficos anterior ao pré-processamento e posterior ao mesmo.

##### Gráfico Cliente Específico Possível Anomalia

In [ ]:
#Especifica o cliente que possui anomalia e cria um gráfico de linha da anomalia
clienteAnomalia = df[df['clientCode'].str.contains("e3322382e75c0d0a8e95f80af703932bd3c38f940aa59a")].sort_values(by='datetime')
sb.lineplot(data=clienteAnomalia, x='datetime', y='pulseCount', hue='clientCode', legend=False)

#### Gráfico Todos os Clientes com essa anomalia específica

In [ ]:
#Verifica todos os clientes que tem as anomalias especificas de ter pulseCount menor que 0 em algum momento
clientesAnomalia = merged_df[merged_df['pulseCount'] < 0]['clientCode'].unique()
clienteAnomalia = merged_df[merged_df['clientCode'].isin(clientesAnomalia)].sort_values(by='datetime')
sb.lineplot(data=clienteAnomalia, x='datetime', y='pulseCount', hue='clientCode', legend=False)

##### Gráfico da Frequência de Pulsos

In [ ]:
# Distribuição da frequência de pulsos
# Moda da frequência de pulsos parece ser 0 de acordo com o gráfico
df.pulseCount.hist(bins=20, figsize=(12,6))
ax = sb.histplot(df.pulseCount, kde=True)
ax.figure.set_size_inches(12,6)
ax.set_title('Distribuição da frequência de pulsos', fontsize=18)
ax.set_xlabel('Consumo', fontsize=10)

##### Gráfico Scatterplot 'pulseCount', 'meterIndex' Sem Processamento

In [ ]:
#Cria um gráfico de dispersão, sem a normalização da coluna pulseCount
ultimo_df_sem_proc = df.groupby(['meterSN', 'clientCode']).last()
sb.scatterplot(data=ultimo_df_sem_proc, x='pulseCount', y='meterIndex', hue='meterSN', legend=False)

##### Gráfico Scatterplot 'pulseCount', 'meterIndex' Com Processamento

In [ ]:
#Cria um gráfico de dispersão com a normalização do pulseCount
ultimo_df_com_proc = merged_df.groupby(['meterSN', 'clientCode']).last()
sb.scatterplot(data=ultimo_df_com_proc, x='pulseCount', y='meterIndex', hue='meterSN', legend=False)

##### Gráfico Boxplot

In [ ]:
#Informações para o gráfico de boxplot; Remove todos que estão com consumo negativo (por questão que vamos perguntar no Sprint Review 2)
#Depois remove outliers que estão fora da regrinha de duas vezes o desvio padrão da média
graficoFinal = df.groupby(['meterSN', 'clientCode']).last()
#graficoFinal = graficoFinal[graficoFinal['pulseCount'] > 0]
graficoFinal.describe()
#graficoFinal = graficoFinal[graficoFinal['pulseCount'] > (graficoFinal.pulseCount.mean() - (2*graficoFinal.pulseCount.std()))]
#graficoFinal = graficoFinal[graficoFinal['pulseCount'] < (graficoFinal.pulseCount.mean() + (2*graficoFinal.pulseCount.std()))]

In [ ]:
# Boxplot da frequência de pulsos
ax = sb.boxplot(x= 'pulseCount', data=graficoFinal, orient='h')
ax.figure.set_size_inches(12,6)
ax.set_title('Boxplot da frequência de pulsos', fontsize=18)
ax.set_xlabel('Consumo', fontsize=14)
ax

##### Gráfico Possível Anomalia

Esse é um gráfico de pulseCount por datetime, específico para clientes que não estão contratando o serviço. Apesar do consumo não estar ativado, estão consumindo de acordo com o gráfico/pulseCount.

In [ ]:
#Gráfico para análise de pessoas que contrataram o sistema mas não teoricamente deveriam estar consumindo
pessoasContratadas = dadosCadastrais[dadosCadastrais.situacao == 'CONTRATADO']['clientCode'].unique()
graficoContratados = df[df['clientCode'].isin(pessoasContratadas)].sort_values(by='datetime')
sb.lineplot(data=graficoContratados, x='datetime', y='pulseCount', hue='clientCode', legend=False)

<h1>Outras possíveis anomalias</h1>
<p>Encontramos mais duas anomalias nesse cliente: O consumo do cliente, em mais de um ponto, diminui, e houve uma série de marcações conseguintes com menos de um minuto de intervalo entre elas.</p>

In [ ]:
#Pega um cliente específico em que a analise foi detectada
clienteAnalise = merged_df[merged_df['clientCode'].str.contains('345b8ca6318576583eb9cb2a1743e725abfdbfcba87f34')]
clienteAnalise.sort_values(by='datetime', inplace=True)
#Calcula a variação do pulseCount, cria uma coluna nova com esse valor
clienteAnalise['variacaoPulseCount'] = clienteAnalise['pulseCount'].diff()
#A primeira linha tem a variação nula, conserta com o valor do pulseCount inicial
clienteAnalise['variacaoPulseCount'].fillna(clienteAnalise['pulseCount'], inplace=True)
clienteAnalise.reset_index(drop=True, inplace=True)
#Cria um gráfico de linha com a variação do pulseCount
sb.lineplot(data=clienteAnalise, x='datetime', y='variacaoPulseCount', legend=False)
#Chama as colunas antes e depois da anomalia
clienteAnalise.loc[clienteAnalise.variacaoPulseCount.idxmin()-1:clienteAnalise.variacaoPulseCount.idxmin()+1]

<h1>Análise de variação:</h1>


In [ ]:
#Garante que os dados estão organizados por data
dadosVariacao = merged_df.sort_values(by='datetime')
#Calcula a variação do pulseCount, cria uma coluna nova com esse valor
dadosVariacao['variacaoPulseCount'] = dadosVariacao.groupby(["meterSN", "clientCode"])['pulseCount'].diff()
#A primeira linha tem a variação nula, conserta com o valor do pulseCount inicial
dadosVariacao['variacaoPulseCount'].fillna(dadosVariacao['pulseCount'], inplace=True)
#Verifica outro cliente com anomalia
variacaoClienteUm = dadosVariacao[dadosVariacao['clientCode'].str.contains('f75e0ef3889a2489f049ebd8acd3066af576f0d012ba8f')]
sb.lineplot(data=variacaoClienteUm, x='datetime', y='variacaoPulseCount', legend=False)

In [ ]:
#Verifica a linha antes e depois da anomalia
variacaoClienteUm.loc[variacaoClienteUm.variacaoPulseCount.idxmin()-1:variacaoClienteUm.variacaoPulseCount.idxmin()+1]